# **Evaluacion de Data set**
Tecnicatura en inteligencia artificial

Universidad Nacional de Hurlingham

Docente: Andrea Rey

Alumno : Nicolas Seivane

Librerias que voy a utilizar


In [24]:
import pandas as pd
import sklearn as sk
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold

# .head()
# .info()
# .isnull().any()
# df.column.value_counts()
# .max()
# .min()
# .describe()

In [25]:
datos = pd.read_csv("C:\\Users\\User\\Documents\\Proyecto integrador\\columna_vertebral\\vertebral_column.csv")
datos.drop_duplicates(inplace=True)
datos.dropna(inplace=True)
#datos.drop(columns=["CLASS"], inplace=True)
#datos['NSP'] = datos['NSP'].replace({1: 'Normal', 2: 'Sospechoso', 3: 'Patologico'})
datos.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Hernia
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Hernia
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Hernia
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Hernia
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Hernia


In [26]:
datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   pelvic_incidence          310 non-null    float64
 1   pelvic_tilt               310 non-null    float64
 2   lumbar_lordosis_angle     310 non-null    float64
 3   sacral_slope              310 non-null    float64
 4   pelvic_radius             310 non-null    float64
 5   degree_spondylolisthesis  310 non-null    float64
 6   class                     310 non-null    object 
dtypes: float64(6), object(1)
memory usage: 17.1+ KB


In [27]:
datos.describe()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000
mean,60.496653,17.542822,51.930930,42.953831,117.920655,26.296694
std,17.236520,10.008330,18.554064,13.423102,13.317377,37.559027
min,26.147921,-6.554948,14.000000,13.366931,70.082575,-11.058179
25%,46.430294,10.667069,37.000000,33.347122,110.709196,1.603727
50%,58.691038,16.357689,49.562398,42.404912,118.268178,11.767934
75%,72.877696,22.120395,63.000000,52.695888,125.467674,41.287352
max,129.834041,49.431864,125.742385,121.429566,163.071041,418.543082


In [28]:
n_filas = len(datos)

for column in datos.columns:
    unicos = datos[column].unique()
    n_unicos = len(unicos)
    
    # Si es categórica (tipo object/category) o es numérica con pocos únicos
    if (datos[column].dtype == 'object') or (len(unicos) <= 10):
        print(f"\nColumna '{column}' (candidata categórica): {n_unicos} valores únicos")
        
        
        for unico in unicos:
                count = len(datos[datos[column] == unico])
                normalizado = count / n_filas
                print(f"   Hay {normalizado:.2%} codificado en {unico}")
    else:
        print(f"\nColumna '{column}' tiene media: {datos[column].mean():.2f}, valor maximo: {datos[column].max():.2f} y valor minimo: {datos[column].min():.2f}")


Columna 'pelvic_incidence' tiene media: 60.50, valor maximo: 129.83 y valor minimo: 26.15

Columna 'pelvic_tilt' tiene media: 17.54, valor maximo: 49.43 y valor minimo: -6.55

Columna 'lumbar_lordosis_angle' tiene media: 51.93, valor maximo: 125.74 y valor minimo: 14.00

Columna 'sacral_slope' tiene media: 42.95, valor maximo: 121.43 y valor minimo: 13.37

Columna 'pelvic_radius' tiene media: 117.92, valor maximo: 163.07 y valor minimo: 70.08

Columna 'degree_spondylolisthesis' tiene media: 26.30, valor maximo: 418.54 y valor minimo: -11.06

Columna 'class' (candidata categórica): 3 valores únicos
   Hay 19.35% codificado en Hernia
   Hay 48.39% codificado en Spondylolisthesis
   Hay 32.26% codificado en Normal


In [29]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        print(column, datos[column].unique())



class ['Hernia' 'Spondylolisthesis' 'Normal']


In [30]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        unicos = datos[column].unique()
        for i in range(len(unicos)):
            datos[column] = datos[column].replace({unicos[i]: i})
        print(column, datos[column].unique())

class [0 1 2]


C:\Users\User\AppData\Local\Temp\ipykernel_5040\495973239.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos[column] = datos[column].replace({unicos[i]: i})


In [31]:
for column in datos.columns:
    if len(datos[column].unique()) > 4:
        datos[column] = (datos[column] - datos[column].mean()) / datos[column].std()

In [32]:
atributos = datos.columns[:-1]
concepto = datos.columns[-1]

datos.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,0.146849,0.500559,-0.664103,-0.184652,-1.445310,-0.706916,0
1,-1.243853,-0.747560,-1.450655,-1.039839,-0.263958,-0.578621,0
2,0.483588,0.467177,-0.099102,0.272642,-0.896237,-0.794137,0
3,0.510564,0.710414,-0.410675,0.125925,-1.205354,-0.401639,0
4,-0.625636,-0.788418,-1.272687,-0.215527,-0.732271,-0.489315,0


In [33]:
def validacion_cruzada(conjunto, atributos, concepto, k=5, random=False, agregar_unos=False, undersample=False, oversample=False):
  
  if undersample:
    cantidad_menor_concepto = conjunto[concepto].value_counts().min()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_menor_concepto, random_state=42) for valor in conjunto[concepto].unique()])

  if oversample:
    cantidad_mayor_concepto = conjunto[concepto].value_counts().max()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_mayor_concepto, replace=True, random_state=42) for valor in conjunto[concepto].unique()])

  if random: conjunto = conjunto.sample(frac=1, random_state=11).reset_index(drop=True)

  x = conjunto[atributos]
  if agregar_unos: x.insert(0, "Unos", 1)

  y = conjunto[concepto]

  resultados = []
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

  for train_index, test_index in skf.split(x, y):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        resultados.append((x_train, y_train, x_test, y_test))

  return resultados

from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)

from sklearn.naive_bayes import GaussianNB

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


from sklearn.svm import SVC

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)

from sklearn.linear_model import LogisticRegression

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)

    y_true_total.extend(y_test)
    y_pred_total.extend(y_pred)

print("Matriz de confusión total:\n", confusion_matrix(y_true_total, y_pred_total))

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

# *Regresion Logistica*

In [34]:
grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1","l2","elasticnet", None],  # 'l1' si usas solver='liblinear' o 'saga'
    "solver": ["lbfgs","saga", "newton-cg"],# según el penalty que uses, liblinear no soporta None, newton-cg no soporta l1 ni elasticnet
    "multi_class": ["ovr", "multinomial"] # si es multiclass
}
 
## cohen_kappa_score,     accuracy_score,    precision_score,     recall_score,    f1_score,    roc_auc_score,


#with open("regresion_logistica_resultados.txt", "w") as f:
with open("regresion_logistica_resultados_Cardiotocografia.txt", "w") as f:
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds):\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for multi_class in grid["multi_class"]:
                for solver in grid["solver"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)
                        
                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                
                        fin = time.time()
                        tiempo_total = fin - inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds) 
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds) undersample:\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for solver in grid["solver"]:
                for multi_class in grid["multi_class"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        tiempo_inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")


                        fin = time.time()
                        tiempo_total = fin - tiempo_inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds)  # Promediar las métricas

                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.w

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)


# *Random Forest*

In [35]:
grid = {
    "n_estimators": [50, 100, 200, 500, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 7, 9],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}



#with open("Random_forest.txt", "w") as f:
with open("Random_forest_Cardiotocografia.txt", "w") as f:
    f.write("Resultados de Random Forest con validacion cruzada (5 folds):\n")

    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Random Forest con validacion cruzada (5 folds) undersample:\n")
    
    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


# *Naive Bayes*

In [36]:
grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  
}

#with open("Naiva_Bayes.txt", "w") as f:
with open("Naiva_Bayes_Cardiotocografia.txt", "w") as f:
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds):\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds) undersample:\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)


# *SVM*

In [37]:
grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 15, 20, 25],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
    "degree": [2,3,4,5,6,7,8,9,10]
}

#with open("SVM.txt", "w") as f:
with open("SVM_Cardiotocografia.txt", "w") as f:
    f.write("Resultados de SVM con validacion cruzada (5 folds):\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                degrees_to_use = grid["degree"] if kernel == "poly" else [3]
                for degree in degrees_to_use:
                        scoring_funcs = {
                                "cohen_kappa": 0,
                                 "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de SVM con validacion cruzada (5 folds) undersample:\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                if kernel != "poly" and "degree" in grid:
                    continue
                for degree in grid["degree"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree if kernel=="poly" else None,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag